## first model

This is the first try (after the proof of concept) to create a model which is capable of detecting all three classes correctly.

In [1]:
import sys
from os.path import join

sys.path.append('..')

raw = join('..', 'data', 'raw')
interim = join('..', 'data', 'interim')
processed = join('..', 'data', 'processed')

from src.training_env import reset_and_populate

reset_and_populate(raw, processed, [300,100,100])

['..\\data\\processed\\train\\n',
 '..\\data\\processed\\validation\\n',
 '..\\data\\processed\\test\\n',
 '..\\data\\processed\\train\\o',
 '..\\data\\processed\\validation\\o',
 '..\\data\\processed\\test\\o',
 '..\\data\\processed\\train\\x',
 '..\\data\\processed\\validation\\x',
 '..\\data\\processed\\test\\x']

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = join(processed, 'train')
validation_dir = join(processed, 'validation')
test_dir = join(processed, 'test')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(32, 32),
    batch_size=20,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(32, 32),
    batch_size=20,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(32, 32),
    batch_size=20,
    class_mode='binary')

Found 900 images belonging to 3 classes.
Found 300 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [3]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Flatten(input_shape=(32, 32, 3)))
model.add(layers.Dense(32, 'relu'))
model.add(layers.Dense(16,'relu'))
model.add(layers.Dense(3, 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                98336     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 51        
Total params: 98,915
Trainable params: 98,915
Non-trainable params: 0
_________________________________________________________________


In [4]:
from tensorflow.keras.optimizers import SGD, RMSprop

sgd = SGD(lr=0.01, decay=1e-5, momentum=0.9, nesterov=False)

model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['acc'])

In [5]:
from tensorflow.keras.callbacks import TensorBoard
import numpy as np

log_dir = join('..', 'logs')

from src.training_env import reset
reset(log_dir)

callbacks = [ TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    embeddings_freq=1) ]

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50,
    callbacks=callbacks)

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
100/100 [==============================] - 8s 75ms/step - loss: 1.0849 - acc: 0.4090 - val_loss: 0.9956 - val_acc: 0.5130
Epoch 2/10
100/100 [==============================] - 6s 62ms/step - loss: 0.9722 - acc: 0.5090 - val_loss: 0.7917 - val_acc: 0.7320
Epoch 3/10
100/100 [==============================] - 6s 60ms/step - loss: 0.5901 - acc: 0.7885 - val_loss: 0.6705 - val_acc: 0.7280
Epoch 4/10
100/100 [==============================] - 6s 64ms/step - loss: 0.5015 - acc: 0.8180 - val_loss: 0.4647 - val_acc: 0.8820
Epoch 5/10
100/100 [==============================] - 7s 67ms/step - loss: 0.4947 - acc: 0.8150 - val_loss: 0.4112 - val_acc: 0.8520
Epoch 6/10
100/100 [==============================] - 6s 64ms/step - loss: 0.3845 - acc: 0.8660 - val_loss: 0.9084 - val_acc: 0.7510
Epoch 7/10
100/100 [==============================] - 7s 67ms/step - loss: 0.5238 - acc: 0.8210 - val_loss: 

In [6]:
model.evaluate_generator(test_generator)

[1.8445007085800171, 0.52666664]

In [7]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

Reusing TensorBoard on port 6006 (pid 16284), started 1 day, 21:12:17 ago. (Use '!kill 16284' to kill it.)